In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from uszipcode import SearchEngine

c:\Users\markm\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Data
The raw data is too big to upload to github. The raw data can be found here: https://drive.google.com/drive/folders/1FO5lCVSy6O_l9hMHG8GRoT3IuasQsT57?usp=share_link

The data will be compiled into one df and then outputted. We will be using the following four states:

TX
CO
NY
WA

## TX

In [2]:
# Import and Clean Up Data
# df_s = df_state
df_s = pd.read_csv('.\Data\TX_EV_Registrations.csv')

# Change to date type
df_s['Registration Date'] = pd.to_datetime(df_s['Registration Date'])
df_s['Registration Date'] = df_s['Registration Date'].dt.to_period('M').dt.to_timestamp()

# Delete rows with error in ZIP Code 
df_s = df_s.loc[df_s['ZIP Code'] != 'Error']

# Convert ZIP Code into int data type
df_s['ZIP Code'] = pd.to_numeric(df_s['ZIP Code'])

FileNotFoundError: [Errno 2] No such file or directory: '.\\Data\\TX_EV_Registrations.csv'

In [ ]:
df_s.head()

,State,ZIP Code,Registration Date,Vehicle Make,Vehicle Model,Vehicle Model Year,Drivetrain Type,Vehicle GVWR Class,Vehicle Category,Vehicle Count,DMV Snapshot (Date),DMV Snapshot ID,Latest DMV Snapshot Flag
0,TX,77014,2021-10-01,TESLA,Model 3,2020.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (1/11/2022),9,False
1,TX,77630,2021-10-01,TESLA,Model 3,2020.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (1/11/2022),9,False
2,TX,78526,2021-10-01,TESLA,Model 3,2020.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (1/11/2022),9,False
3,TX,78045,2021-10-01,TESLA,Model 3,2020.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (1/11/2022),9,False
4,TX,77057,2021-10-01,TESLA,Model 3,2020.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (1/11/2022),9,False


In [ ]:
df = df_s.groupby(['ZIP Code', 'Registration Date', 'Drivetrain Type']).agg({'Vehicle Count': 'sum'})
# Reset the index to include 'Category' as a column in the DataFrame
df = df.reset_index()
df.loc[df['ZIP Code'] == 73301]

,ZIP Code,Registration Date,Drivetrain Type,Vehicle Count
0,73301,2021-03-01,PHEV,15
1,73301,2021-06-01,PHEV,17


In [ ]:
df.insert(loc=0, column='State', value='TX')


In [ ]:
df
# 48469 before trimming

# create a search engine instance
search = SearchEngine()

# search for zip codes 
zips = search.by_state("TX", returns=100000)

# Extract only TX zip codes
df = df[df['ZIP Code'].isin(zips)]
df

# 48469 after

# There are 3 zip codes seen from Tableau that are in OK
ok_z=[74748, 74034, 73749]

mask = ~df['ZIP Code'].isin(ok_z)
df = df[mask]
df
# 48465 after removal

Download C:\Users\markm\.uszipcode\simple_db.sqlite from https://github.com/MacHu-GWU/uszipcode-project/releases/download/1.0.1.db/simple_db.sqlite ...
  1.00 MB downloaded ...
  2.00 MB downloaded ...
  3.00 MB downloaded ...
  4.00 MB downloaded ...
  5.00 MB downloaded ...
  6.00 MB downloaded ...
  7.00 MB downloaded ...
  8.00 MB downloaded ...
  9.00 MB downloaded ...
  10.00 MB downloaded ...
  11.00 MB downloaded ...
  Complete!


,State,ZIP Code,Registration Date,Drivetrain Type,Vehicle Count


In [ ]:
# This is to double check that the data is being analyzed correctly in Tableau
# I randomly picked ZIP 75002 that had a sum of vehicle counts of 11733
# Extract rows that corresponding to 75002 and sum up the vehicle count to verify
df_z = df_s.loc[df_s['ZIP Code'] == 75002] #11733
sum(df_z['Vehicle Count'])


11733

## WA

In [ ]:
# Import and Clean Up Data
df_s = pd.read_csv('.\Data\wa_ev_registrations_public.csv')

# Convert ZIP Code into int data type
df_s['ZIP Code'] = pd.to_numeric(df_s['ZIP Code'], errors='coerce')
df_s = df_s.dropna()
df_s['ZIP Code'] = df_s['ZIP Code'].astype(int)

# Change to date type and change dates to the first of the month
df_s['Registration Valid Date'] = pd.to_datetime(df_s['Registration Valid Date'])
df_s['Registration Valid Date'] = df_s['Registration Valid Date'].dt.to_period('M').dt.to_timestamp()

# Add a vehicle count column
df_s['Vehicle Count'] = 1

# search for zip codes 
zips = search.by_state("WA", returns=100000)

# Extract only rows in zips
df_s = df_s[df_s['ZIP Code'].isin(zips)]
df_s

C:\Users\markm\AppData\Local\Temp\ipykernel_20516\3333471911.py:2: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_s = pd.read_csv('.\Data\wa_ev_registrations_public.csv')


,DMV ID,DMV Snapshot,ZIP Code,State,Registration Valid Date,DMV ID Complete,Registration Expiration Date,VIN Prefix,VIN Model Year,Vehicle Name,Technology,Vehicle Count


In [ ]:
df_s.head()

,DMV ID,DMV Snapshot,ZIP Code,State,Registration Valid Date,DMV ID Complete,Registration Expiration Date,VIN Prefix,VIN Model Year,Vehicle Name,Technology,Vehicle Count


In [ ]:
# Extract temp df before concatenating to main

df_t = df_s.groupby(['ZIP Code', 'Registration Valid Date', 'Technology']).agg({'Vehicle Count': 'sum'})
# Reset the index to include 'Category' as a column in the DataFrame
df_t = df_t.reset_index()
#df_t.loc[df_t['ZIP Code'] == 73301]
print(max(df_t['Vehicle Count']))
print(sum(df_t['Vehicle Count']))

df_t = df_t.rename(columns={'Registration Valid Date': 'Registration Date', 'Technology' : 'Drivetrain Type'})

ValueError: max() arg is an empty sequence

In [ ]:
df_t.insert(loc=0, column='State', value='WA')
df_t.head()

,State,ZIP Code,Registration Date,Drivetrain Type,Vehicle Count
0,WA,98001,2011-05-01,BEV,4
1,WA,98001,2011-06-01,BEV,2
2,WA,98001,2011-09-01,BEV,4
3,WA,98001,2011-10-01,BEV,2
4,WA,98001,2011-12-01,BEV,1


In [ ]:
df = pd.concat([df, df_t], axis=0)

In [ ]:
df

,State,ZIP Code,Registration Date,Drivetrain Type,Vehicle Count
0,TX,73301,2021-03-01,PHEV,15
1,TX,73301,2021-06-01,PHEV,17
2,TX,73354,2021-06-01,PHEV,3
6,TX,74409,2022-03-01,BEV,10
7,TX,74493,2021-11-01,BEV,11
...,...,...,...,...,...
33520,WA,99402,2017-06-01,BEV,1
33521,WA,99402,2017-07-01,BEV,2
33522,WA,99402,2018-04-01,PHEV,2
33523,WA,99402,2018-06-01,BEV,1


## CO

In [ ]:
# Import and Clean Up Data
df_s = pd.read_csv('.\Data\CO_EV_Registrations.csv')

# Convert ZIP Code into int data type
df_s['ZIP Code'] = pd.to_numeric(df_s['ZIP Code'], errors='coerce')
df_s = df_s.dropna()
df_s['ZIP Code'] = df_s['ZIP Code'].astype(int)

# Change to date type and change dates to the first of the month
df_s['Registration Date'] = pd.to_datetime(df_s['Registration Date'])
df_s['Registration Date'] = df_s['Registration Date'].dt.to_period('M').dt.to_timestamp()

# search for zip codes 
zips = search.by_state("CO", returns=100000)

# Extract only rows in zips
df_s = df_s[df_s['ZIP Code'].isin(zips)]


C:\Users\markm\AppData\Local\Temp\ipykernel_20516\4175660473.py:2: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_s = pd.read_csv('.\Data\CO_EV_Registrations.csv')


In [ ]:
df_s.head()

,State,ZIP Code,Registration Date,Vehicle Make,Vehicle Model,Vehicle Model Year,Drivetrain Type,Vehicle GVWR Class,Vehicle Category,Vehicle Count,DMV Snapshot (Date),DMV Snapshot ID,Latest DMV Snapshot Flag
0,CO,80241,2021-10-01,TESLA,Model 3,2019.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (10/01/2022),45,False
1,CO,80005,2021-10-01,TESLA,Model 3,2019.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (10/01/2022),45,False
2,CO,80112,2021-10-01,TESLA,Model 3,2019.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (10/01/2022),45,False
3,CO,80424,2021-10-01,TESLA,Model 3,2019.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (10/01/2022),45,False
4,CO,80223,2021-10-01,TESLA,Model 3,2019.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (10/01/2022),45,False


In [ ]:
df_s['ZIP Code']

0          80241
1          80005
2          80112
3          80424
4          80223
           ...  
1397572    80020
1397573    80020
1397574    80020
1397575    80020
1397576    80020
Name: ZIP Code, Length: 1387702, dtype: int32

In [ ]:
# Extract temp df before concatenating to main

df_t = df_s.groupby(['ZIP Code', 'Registration Date', 'Drivetrain Type']).agg({'Vehicle Count': 'sum'})
# Reset the index to include 'Category' as a column in the DataFrame
df_t = df_t.reset_index()
df_t.insert(loc=0, column='State', value='CO')
df = pd.concat([df, df_t], axis=0)
df

,State,ZIP Code,Registration Date,Drivetrain Type,Vehicle Count
0,TX,73301,2021-03-01,PHEV,15
1,TX,73301,2021-06-01,PHEV,17
2,TX,73354,2021-06-01,PHEV,3
6,TX,74409,2022-03-01,BEV,10
7,TX,74493,2021-11-01,BEV,11
...,...,...,...,...,...
43043,CO,81657,2023-01-01,BEV,30
43044,CO,81657,2023-01-01,PHEV,8
43045,CO,81657,2023-02-01,BEV,21
43046,CO,81657,2023-02-01,PHEV,1


## NY

In [ ]:
# Import and Clean Up Data
fpth = os.path.join('.\Data', 'NY_EV_Registrations.csv')
df_s = pd.read_csv(fpth)

# Convert ZIP Code into int data type
df_s['ZIP Code'] = pd.to_numeric(df_s['ZIP Code'], errors='coerce')
df_s = df_s.dropna()
df_s['ZIP Code'] = df_s['ZIP Code'].astype(int)

# Change to date type and change dates to the first of the month
df_s['Registration Date'] = pd.to_datetime(df_s['Registration Date'])
df_s['Registration Date'] = df_s['Registration Date'].dt.to_period('M').dt.to_timestamp()

# search for zip codes 
zips = search.by_state("NY", returns=100000)

# Extract only rows in zips
df_s = df_s[df_s['ZIP Code'].isin(zips)]

C:\Users\markm\AppData\Local\Temp\ipykernel_20516\1469016851.py:3: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_s = pd.read_csv(fpth)


In [ ]:
df_s.head()

,State,ZIP Code,Registration Date,Vehicle Make,Vehicle Model,Vehicle Model Year,Drivetrain Type,Vehicle GVWR Class,Vehicle Category,Vehicle Count,DMV Snapshot (Date),DMV Snapshot ID,Latest DMV Snapshot Flag
0,NY,10576,2019-09-01,TESLA,Model 3,2019.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (9/2/2020),42,False
1,NY,11694,2019-09-01,TESLA,Model 3,2019.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (9/2/2020),42,False
2,NY,10469,2019-09-01,TESLA,Model 3,2019.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (9/2/2020),42,False
3,NY,12061,2019-09-01,TESLA,Model 3,2019.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (9/2/2020),42,False
4,NY,14059,2019-09-01,TESLA,Model 3,2019.0,BEV,1,Light-Duty (Class 1-2A),1,DMV Snapshot (9/2/2020),42,False


In [ ]:
min(df_s['ZIP Code'])

646

In [ ]:
# Extract temp df before concatenating to main

df_t = df_s.groupby(['ZIP Code', 'Registration Date', 'Drivetrain Type']).agg({'Vehicle Count': 'sum'})
# Reset the index to include 'Category' as a column in the DataFrame
df_t = df_t.reset_index()
df_t.insert(loc=0, column='State', value='NY')
df = pd.concat([df, df_t], axis=0)
df

,State,ZIP Code,Registration Date,Drivetrain Type,Vehicle Count
0,TX,73301,2021-03-01,PHEV,15
1,TX,73301,2021-06-01,PHEV,17
2,TX,73354,2021-06-01,PHEV,3
6,TX,74409,2022-03-01,BEV,10
7,TX,74493,2021-11-01,BEV,11
...,...,...,...,...,...
113342,NY,14905,2022-12-01,PHEV,6
113343,NY,14905,2023-01-01,BEV,5
113344,NY,14905,2023-01-01,PHEV,6
113345,NY,14905,2023-02-01,BEV,1


# Output CSV

In [ ]:
df = df.reset_index(drop=True)
df.to_csv('.\output\TX_WA_CO_NY.csv')